<a href="https://colab.research.google.com/github/jpbjoaopedro/agente-ia-atendimento-suporte/blob/main/Atendimento_e_Suporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tecnologias utilizadas

* RAG e Histórico do chat - pipeline com LangChain;
---
LLM com recuperação de informação externas em tempo real, podendo ser sites, documentos, etc

* Busca vetorial - FAISS (banco de dados vetorial)
* Parsing / pré-processamento de documentos - PyMuPDF
* Streamlit - UI

## RAG

Indexão - Ingestão de dados, indexação e armazenamento em vector database

Recuperação e geração - Processa o pedido e recupera dados relevantes do índice, depois passando-os para o modelo usar como contexto.

1. Carregar
2. Dividir
3. Armazenar
4. Recuperar
5. Gerar

# Sistema

## Instalação das bibliotecas

In [ ]:
!pip install -q langchain langchain-groq langchain-community langchain-huggingface
!pip install -q faiss-cpu sentence-transformers PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 21.4 MB/s eta 0:00:00


## Importações

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import getpass

## Carregando LLM

In [ ]:
os.environ['GROQ_API_KEY'] = getpass.getpass()

··········


In [ ]:
def load_llm(id_model, temparature):
  llm = ChatGroq(
      model = id_model,
      temperature = temparature,
      max_tokens = None,
      timeout = None,
      max_retries = 2
  )

  return llm

In [ ]:
id_model = "llama3-70b-8192" # @param {type: "string"}
temperature = 0.7 # @param {type: "slider", min: 0.1, max: 1.5, step: 0.1}

llm = load_llm(id_model, temperature)

In [ ]:
prompt = "Olá! como posso alterar minha senha?" # @param {type: "string"}

template = ChatPromptTemplate.from_messages([
    ("system", "você é um assistente virtual prestativo e está respondendo perguntas gerais"),
    ("human", "{prompt}"),
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
res

'Olá!\n\nAlterar a senha é um processo simples e importante para manter a segurança da sua conta. O passo a passo para alterar a senha pode variar dependendo do sistema ou plataforma que você está usando. Aqui vão algumas opções comuns:\n\n**Se você está usando um site ou aplicativo:**\n\n1. Faça login na sua conta com seu nome de usuário e senha atuais.\n2. Clique no ícone de perfil ou na opção "Conta" ou "Perfil" geralmente localizado no canto superior direito da tela.\n3. Selecione a opção "Alterar senha" ou "Senha" e, em seguida, "Alterar".\n4. Digite sua senha atual e, em seguida, insira a nova senha desejada. Certifique-se de que atenda aos requisitos de segurança, como tamanho mínimo e caracteres especiais.\n5. Confirme a nova senha e clique em "Salvar" ou "Atualizar".\n\n**Se você está usando um dispositivo móvel:**\n\n1. Abra o aplicativo ou vá para as configurações do dispositivo.\n2. Toque no ícone de perfil ou na opção "Conta" ou "Perfil".\n3. Selecione a opção "Alterar sen

In [ ]:
def show_res(res):
  from IPython.display import Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()
  else:
    res = res.strip()
  display(Markdown(res))

show_res(res)

Olá!

Alterar a senha é um processo simples e importante para manter a segurança da sua conta. O passo a passo para alterar a senha pode variar dependendo do sistema ou plataforma que você está usando. Aqui vão algumas opções comuns:

**Se você está usando um site ou aplicativo:**

1. Faça login na sua conta com seu nome de usuário e senha atuais.
2. Clique no ícone de perfil ou na opção "Conta" ou "Perfil" geralmente localizado no canto superior direito da tela.
3. Selecione a opção "Alterar senha" ou "Senha" e, em seguida, "Alterar".
4. Digite sua senha atual e, em seguida, insira a nova senha desejada. Certifique-se de que atenda aos requisitos de segurança, como tamanho mínimo e caracteres especiais.
5. Confirme a nova senha e clique em "Salvar" ou "Atualizar".

**Se você está usando um dispositivo móvel:**

1. Abra o aplicativo ou vá para as configurações do dispositivo.
2. Toque no ícone de perfil ou na opção "Conta" ou "Perfil".
3. Selecione a opção "Alterar senha" ou "Senha" e, em seguida, "Alterar".
4. Digite sua senha atual e, em seguida, insira a nova senha desejada. Certifique-se de que atenda aos requisitos de segurança.
5. Confirme a nova senha e toque em "Salvar" ou "Atualizar".

**Se você estiver tendo problemas para alterar a senha:**

* Verifique se você está conectado à internet e se sua conta está ativa.
* Tente fechar e reabrir o aplicativo ou site.
* Se você esqueceu sua senha atual, você pode precisar redefini-la seguindo as instruções de redefinição de senha do sistema ou plataforma.

Lembre-se de que é importante manter sua senha segura e não compartilhá-la com ninguém. Se tiver alguma dúvida adicional, sinta-se à vontade para perguntar!

## Definindo contexto

In [ ]:
context = """
Para alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.
Para alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.
"""

prompt = f"""
Como alterar minha senha?

Contexto: {context}
"""

In [ ]:
res = chain.invoke({"prompt": prompt})
show_res(res)

Para alterar sua senha, você tem duas opções:

**No aplicativo:**

1. Clique no menu "Minha conta"
2. Selecione "Alterar senha"

**No site:**

1. Acesse "Configurações" no menu do topo
2. Selecione "Minha conta"
3. Clique em "Alterar senha"

Siga esses passos e você poderá alterar sua senha com facilidade!

## Prompt para RAG

In [ ]:
template_rag = """
Pergunta: {input}
Contexto: {context}
"""

In [ ]:
from langchain.prompts import PromptTemplate

prompt_rag = PromptTemplate.from_template(template_rag)
print(prompt_rag)

input_variables=['context', 'input'] input_types={} partial_variables={} template='\nPergunta: {input}\nContexto: {context}\n'


In [ ]:
chain_rag = prompt_rag | llm | StrOutputParser()

input = "Como alterar minha senha?"

res = chain_rag.invoke({"input": input, "context": context})

show_res(res)

Para alterar sua senha, existem dois processos dependendo de onde você está acessando:

**No aplicativo:**

1. Clique no menu "Minha conta".
2. Selecione "Alterar senha".

**No site:**

1. Acesse "Configurações" no menu do topo.
2. Selecione "Minha conta".
3. Clique em "Alterar senha".

Siga esses passos para atualizar sua senha com segurança!

## Etapas Indexação

### 1 - Carregar o conteúdo

In [ ]:
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
print(f"Arquivo carregado: {file_path}")

Saving manual-safebank.pdf to manual-safebank.pdf
Arquivo carregado: manual-safebank.pdf


In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "manual-safebank.pdf"

loader = PyMuPDFLoader(file_path)
doc = loader.load()
doc[0]

Document(metadata={'producer': 'Skia/PDF m137 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'manual-safebank.pdf', 'file_path': 'manual-safebank.pdf', 'total_pages': 11, 'format': 'PDF 1.4', 'title': 'manual-safebank', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Manual de Atendimento e Uso – SafeBank \n \n1. Introdução \nObjetivo do Manual  \nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento \ncompleto do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, \naté procedimentos de segurança, suporte e resolução de problemas. O documento também \nserve como material de consulta para dúvidas frequentes. \nVisão geral do banco digital  \nNosso banco digital oferece serviços bancários completos de forma 100% online, por meio \ndo aplicativo e do site. Entre os principais serviços estão conta digital gratuita, \ntransferênc

In [ ]:
import pprint
pprint.pp(doc[0].metadata)

{'producer': 'Skia/PDF m137 Google Docs Renderer',
 'creator': '',
 'creationdate': '',
 'source': 'manual-safebank.pdf',
 'file_path': 'manual-safebank.pdf',
 'total_pages': 11,
 'format': 'PDF 1.4',
 'title': 'manual-safebank',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '',
 'trapped': '',
 'modDate': '',
 'creationDate': '',
 'page': 0}


In [ ]:
len(doc[0].page_content)

1668

In [ ]:
print(doc[0].page_content[:1000])

Manual de Atendimento e Uso – SafeBank 
 
1. Introdução 
Objetivo do Manual  
Este manual tem como objetivo orientar clientes e atendentes sobre o funcionamento 
completo do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, 
até procedimentos de segurança, suporte e resolução de problemas. O documento também 
serve como material de consulta para dúvidas frequentes. 
Visão geral do banco digital  
Nosso banco digital oferece serviços bancários completos de forma 100% online, por meio 
do aplicativo e do site. Entre os principais serviços estão conta digital gratuita, 
transferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e 
muito mais, sempre com foco em praticidade, segurança e transparência. 
Público-alvo  
O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções 
financeiras modernas, acessíveis e com gestão 100% digital.  
 
2. Cadastro e Abertura de Conta 
Requisitos para abertura de conta 

In [ ]:
def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  doc = loader.load()
  content = "\n".join([page.page_content for page in doc])

  return content

In [ ]:
extract_text_pdf(file_path)

'Manual de Atendimento e Uso – SafeBank \n \n1. Introdução \nObjetivo do Manual  \nEste manual tem como objetivo orientar clientes e atendentes sobre o funcionamento \ncompleto do banco digital, abordando desde o acesso à conta, funcionalidades e serviços, \naté procedimentos de segurança, suporte e resolução de problemas. O documento também \nserve como material de consulta para dúvidas frequentes. \nVisão geral do banco digital  \nNosso banco digital oferece serviços bancários completos de forma 100% online, por meio \ndo aplicativo e do site. Entre os principais serviços estão conta digital gratuita, \ntransferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e \nmuito mais, sempre com foco em praticidade, segurança e transparência. \nPúblico-alvo  \nO banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções \nfinanceiras modernas, acessíveis e com gestão 100% digital.  \n \n2. Cadastro e Abertura de Conta \nRequisitos para

In [ ]:
docs_path = Path("/content")
pdf_files = [f for f in docs_path.glob("*.pdf")]
pdf_files

[PosixPath('/content/manual-safebank.pdf')]

In [ ]:
loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]
len(pdf_files)

1

### 2 - Divisão em pedaços de texto / Split

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
chunks = []

for doc in loaded_documents:
  chunks.extend(text_splitter.split_text(doc))

print(f"Numero de chunks criados: {len(chunks)}")

Numero de chunks criados: 44


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# embedding_model = "sentence-transformers/all-mpnet-base-v2"
embedding_model = "BAAI/bge-m3"

embeddings = HuggingFaceEmbeddings(model_name = embedding_model)

input = "Um teste"

result = embeddings.embed_query(input)
len(result)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

1024

In [ ]:
print(result)

[0.002311482559889555, 0.029925554990768433, -0.06028471514582634, 0.0007638970855623484, -0.0015801790868863463, -0.017279334366321564, 0.013361066579818726, 0.033754944801330566, 0.011608694680035114, 0.0013604372506961226, -0.002627710113301873, 0.008935314603149891, -0.019280044361948967, -0.002665887353941798, 0.046379975974559784, -0.030164139345288277, 0.0016119158826768398, -0.02057412639260292, -0.012764684855937958, -0.013486174866557121, -0.02407396212220192, -0.041337382048368454, 0.06570088863372803, -0.0003265820851083845, -0.00134829874150455, 0.01719357818365097, -0.03539242967963219, -0.01723906584084034, 0.02015024796128273, 0.04997102543711662, 0.018024992197752, -0.012557161040604115, 0.0016263751313090324, -0.03557245433330536, -0.037849102169275284, -0.021105295047163963, 0.013232960365712643, -0.05502304807305336, -0.026066238060593605, 0.010389315895736217, 0.04742942377924919, -0.023233359679579735, -0.00445597292855382, -0.05877715349197388, -0.014002990908920

### 3 - Armazenamento

In [ ]:
len(chunks)

44

In [ ]:
vectorstore = FAISS.from_texts(chunks, embedding = embeddings)

In [ ]:
vectorstore.save_local("index_faiss")

In [ ]:
#db = FAISS.load_local("index_faiss", embeddings)

### 4 - Configurando o recuperador de texto / Retriever

In [ ]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs = {"k": 6})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x78ed71edbc20>, search_kwargs={'k': 6})

### 5 - Geração

In [ ]:
prompt_rag = PromptTemplate(
    input_variables=["context", "input"],
    template=template_rag,
)
prompt_rag

PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nPergunta: {input}\nContexto: {context}\n')

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain_rag = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [ ]:
res = chain_rag.invoke("how can i ")
show_res(res)

It seems like you're trying to ask a question, but it got cut off!

Please feel free to complete your question, and I'll do my best to help you based on the context provided.

## Melhorias no prompt

In [ ]:
system_prompt = """
Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os servições de uma empresa
Use os seguintes pedaços de contexto para responder à pergunta do usuário.
Se você não souber a resposta, apenas diga que não sabe, não tente inventar uma resposta.
Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
Mantenha a resposta concisa.
Responda em português. \n\n
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Pergunta: {input}\n\nContexto: {context}"),
    ]
)

qa_prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nVocê é um assistente virtual prestativo e está respondendo perguntas gerais sobre os servições de uma empresa\nUse os seguintes pedaços de contexto para responder à pergunta do usuário.\nSe você não souber a resposta, apenas diga que não sabe, não tente inventar uma resposta.\nMas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.\nMantenha a resposta concisa.\nResponda em português. \n\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='Pergunta: {input}\n\nContexto: {context}'), additional_kwargs={})])

## Melhoria no método de busca

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 3, 'fetch_k': 4}
)

## Pipeline RAG avançada

### 1 - Reformulação da consulta para contextualizar

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
context_q_system_prompt = context_q_system_prompt
context_q_user_prompt = "Question: {input}"
context_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", context_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", context_q_user_prompt)
    ]
)

### 2 - Chain para contextualização

In [ ]:
history_aware_retriever = create_history_aware_retriever(
    llm = llm, retriever = retriever, prompt = context_q_prompt
)

### 3 - Chain para perguntas e respostas (Q&A)

In [ ]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "Pergunta: {input}\n\n Contexto: {context}"),
    ]
)

qa_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

def config_rag_chain(llm, retriever):

    # Prompt de contextualização
    context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

    context_q_system_prompt = context_q_system_prompt
    context_q_user_prompt = "Question: {input}"
    context_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", context_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", context_q_user_prompt),
        ]
    )

    # Chain para contextualização
    history_aware_retriever = create_history_aware_retriever(
        llm=llm, retriever=retriever, prompt=context_q_prompt
    )

    # Prompt para perguntas e respostas (Q&A)
    system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
    Use os seguintes pedaços de contexto recuperado para responder à pergunta.
    Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
    Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
    Mantenha a resposta concisa.
    Responda em português. \n\n"""

    qa_prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "Pergunta: {input}\n\n Contexto: {context}"),
    ])

    # Configurar LLM e Chain para perguntas e respostas (Q&A)

    qa_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(
        history_aware_retriever,
        qa_chain,
    )

    return rag_chain

## Adicionando histórico de conversas

In [ ]:
store = {}

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

In [ ]:
qa_chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_session_history,
    input_messages_key = "input",
    history_messages_key = "chat_history"
)

In [ ]:
session_id = "usuario_42"

In [ ]:
response1 = qa_chain_with_history.invoke(
    {"input": "Olá! como posso alterar minha senha?", "context": ""},
    config = {"configurable": {"session_id": session_id}}
)

print(response1)

Olá! Para alterar sua senha, basta acessar a página de login da nossa plataforma, clicar em "Esqueceu sua senha" e seguir as instruções para redefinir sua senha.


In [ ]:
response2 = qa_chain_with_history.invoke(
    {"input": "Qual foi a minha última pergunta?", "context": ""},
    config = {"configurable": {"session_id": session_id}}
)

print(response2)

Sua última pergunta foi "Olá! como posso alterar minha senha?"


## Passando histórico de chat à Chain

In [ ]:
rag_chain = config_rag_chain(llm, retriever)

In [ ]:
chat_history = []

In [ ]:
input = "Qual foi minha ultima pergunta?"

# 1
chat_history.append(HumanMessage(content=input))

# 2
result = rag_chain.invoke({"input": input, "chat_history": chat_history})

res = result['answer']
print(res)

# 3
chat_history.append(AIMessage(content=res))

Sua última pergunta foi "Como alterar minha senha?"


In [ ]:
chat_history

[HumanMessage(content='como alterar minha senha?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Para alterar sua senha, selecione "Minha conta" e "Alterar senha". O sistema pedirá a senha atual e, por segurança, não permitirá reutilizar senhas anteriores recentes.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Qual foi minha ultima pergunta?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Sua última pergunta foi "Como alterar minha senha?"', additional_kwargs={}, response_metadata={})]

## Configuração da função de Indexação e Recuperação

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def config_retriever(folder_path="/content"):
  # Carregar documentos
  docs_path = Path("/content")
  pdf_files = [f for f in docs_path.glob("*.pdf")]

  loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]

  # Divisão em pedaços de texto / Split
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1000,
      chunk_overlap = 200
  )
  chunks = []

  for doc in loaded_documents:
    chunks.extend(text_splitter.split_text(doc))

  # Embeddings
  embedding_model = "BAAI/bge-m3"

  embeddings = HuggingFaceEmbeddings(model_name = embedding_model)

  # Armazenamento
  vectorstore = FAISS.from_texts(chunks, embedding = embeddings)

  vectorstore.save_local('index_faiss')

  # Configurando o recuperador de texto / Retriever
  retriever = vectorstore.as_retriever(
      search_type='mmr',
      search_kwargs={'k':3, 'fetch_k': 4}
  )

  return retriever

In [ ]:
retriever = config_retriever()

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

def config_rag_chain(llm, retriever):

  # Prompt de contextualização
  context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

  context_q_system_prompt = context_q_system
  context_q_user_prompt = "Question: {input}"
  context_q_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", context_q_system_prompt),
          MessagesPlaceholder("chat_history"),
          ("human", context_q_user_prompt),
      ]
  )

  # Chain para contextualização
  history_aware_retriever = create_history_aware_retriever(
      llm=llm, retriever=retriever, prompt=context_q_prompt
  )

  # Prompt para perguntas e respostar (Q&A)
  system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
  Use os seguintes pedaços de contexto recuperado para responder à pergunta.
  Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
  Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
  Responda em português. \n\n"""

  qa_prompt = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      MessagesPlaceholder("chat_history"),
      ("human", "Pergunta: {input}\n\n Contexto: {context}"),
  ])


  # Configurar LLM e Chain para perguntas e respostas (Q&Ä)

  qa_chain = create_stuff_documents_chain(llm, qa_prompt)

  rag_chain = create_retrieval_chain(
      history_aware_retriever,
      qa_chain
  )

  return rag_chain

## Junção da pipeline final + Conclusão

In [ ]:
def chat_llm(rag_chain, input, chat_history):
  chat_history.append(HumanMessage(content = input))

  response = rag_chain.invoke({
      "input": input,
      "chat_history": chat_history
  })

  res = response['answer']

  chat_history.append(AIMessage(content = res))

  return chat_history

In [ ]:
chat_history = []

In [ ]:
input = "Olá! Tudo bem" # @param {type: "string"}

chat_history = chat_llm(rag_chain, input, chat_history)

In [ ]:
chat_history

[HumanMessage(content='Olá! Tudo bem', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Olá! Sim, tudo bem! Estou aqui para ajudar com qualquer dúvida sobre os serviços da Safe Bank. O que você precisa saber ou perguntar?', additional_kwargs={}, response_metadata={})]

## Interface com streamlit

In [ ]:
!pip install -q streamlit python-dotenv
!npm install -q localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇
up to date, audited 23 packages in 1s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠇

# Conclusão projeto

In [ ]:
%%writefile .env

GROQ_API_KEY = userdata.get('GROQ_API_KEY')

Overwriting .env


In [ ]:
%%writefile app02.python
import streamlit as st
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
from langchain_community.document_loaders import PyMuPDF
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()

st.set_page_config(page_title="Atendimento SafeBank 🤖", page_icon="🤖")
st.title("Atendimento SafeBank")

id_model = "deepseek-r1-distill-llama-70b"
temperature = 0.7
path = "/content"

### Carregamento da LLM
def load_llm(id_model, temperature):
  llm = ChatGroq(
    model = id_model,
    temparature = temperature,
    max_tokens = None
    timeout = None,
    max_retries = 2
  )
  return llm

llm = load_llm(id_model, temperature)

### Exibição do resultado
def show_res(res):
  from IPython.display import Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()
  else:
    res = res.strip()
  display(Markdown(res))

### Extração do conteúdo
def extract_text_pdf(file_path):
  loader = PyMuPDFLoader(file_path)
  doc = loader.load()
  content = "\n".join([page.page_content for page in doc])
  return content

### Indexação e recuperação
def config_retriever(folder_path="/contet"):
  # Carregar documentos
  docs_path = Path(folder_path)
  pdf_files = [f for f in docs_path.glob("*.pdf")]

  if len(pdf_files) < 1:
    st.error("Nenhum arquivo PDF carregado")
    st.stop()

  loaded_documents = [extract_text_pdf(pdf) for pdf in pdf_files]

  # Divisão em pedaços de texto / Split
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1000,
      chunk_overlap = 200
  )

  chunks = []
  for doc in loaded_documents:
    chunks.extend(text_splitter.split_text(doc))

  # Embeddings
  embedding_model = "BAAI/bge-m3"

  embeddings = HuggingFaceEmbeddings(model_name = embedding_model)

  # Armazenamento
  vectorstore = FAISS.from_texts(chunks, embedding = embeddings)

  vectorstore.save_local('index_faiss')

  # Configurando o recuperador de texto / Retriever
  retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k':3, 'fetch_k': 4}
  )

  return retriever


### Chain da RAG
def config_rag_chain(llm, retriever):

  # Prompt de contextualização
  context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."

  context_q_system_prompt = context_q_system_prompt
  context_q_user_prompt = "Question: {input}"
  context_q_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", context_q_system_prompt),
          MessagesPlaceholder("chat_history"),
          ("human", context_q_user_prompt)
      ]
  )

  # Chain para contextualização
  history_aware_retriever = create_history_aware_retriever(
      llm=llm, retriever=retriever, prompt=context_q_prompt
  )

  # Prompt para perguntas e respostar (Q&A)
  system_prompt = """Você é um assistente virtual prestativo e está respondendo perguntas gerais sobre os serviços de uma empresa.
  Use os seguintes pedaços de contexto recuperado para responder à pergunta.
  Se você não sabe a resposta, apenas comente que não sabe dizer com certeza.
  Mas caso seja uma dúvida muito comum, pode sugerir como alternativa uma solução possível.
  Responda em português. \n\n"""

  qa_prompt = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      MessagesPlaceholder("chat_history"),
      ("human", "Pergunta: {input}\n\n Contexto: {context}"),
  ])


  # Configurar LLM e Chain para perguntas e respostas (Q&Ä)

  qa_chain = create_stuff_documents_chain(llm, qa_prompt)

  rag_chain = create_retrieval_chain(
      history_aware_retriever,
      qa_chain
  )

  return rag_chain

### Interação com chat
def chat_llm(rag_chain, input):
  st.session_state.chat_history.append(HumanMessage(content=input))

  response = rag_chain.invoke({
    "input": input,
    "chat_history": st.session_state.chat_history
  })

  res = response['answer']
  res = res.split("</think>")[-1].strip() if "</think>" in res else res.strip()

  st.session_state.chat_history.append(AIMessage(content=res))

  return res

input = st.chat_input("Digite sua mensagem aqui...")

if "chat_history" not in st.session_state:
  st.session_state.chat_history = [
    AIMessage(content = "Olá, sou o seu assistente virtual! Como posso te ajudar?")
  ]

if "retriever" not in st.session_state:
  st.session_state.retriever = None

for message in st.session_state.chat_history:
  if isinstance(message, AIMessage):
    with st.chat_message("AI"):
      st.write(message.content)
  elseif isinstance(message, HumanMessage):
    with st.chat_message("Human"):
      st.write(message.content)

if input is not None:
  with st.chat_message("Human"):
    st.markdown(input)

  with st_chat_message("AI"):
    if st.session_state.retriever is None:
      st.session_state.retriever = config_retriever(path)
    rag_chain = config_rag_chain(llm, st.session_state.retriever)
    res = chat_llm(rag_chain,input)
    st.write(res)

Overwriting app02.python


## Execução do Streamlit

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

!ngrok config add-authtoken 31pu62sjZEeUm4pPEmyFCBx9M9z_zwrCzFcbpmBAuziUe4Pj
!streamlit run app02.py --server.port 8501 &>/content/logs.txt &

public_url = ngrok.connect(8501)
public_url

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


<NgrokTunnel: "https://8fb231b65afc.ngrok-free.app" -> "http://localhost:8501">